<a href="https://colab.research.google.com/github/facuarre96/ChlorineProject/blob/main/scrapping_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#set up + import
!pip install pandas openpyxl requests beautifulsoup4 PyMuPDF
import pandas as pd
from bs4 import BeautifulSoup
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import html
import fitz
import numpy as np
from google.colab import drive

In [ ]:
#more set up + import
drive.mount('/content/drive')

file_path2 = '/content/drive/MyDrive/1-s2.0-S027323002200174X-mmc2.xlsx'
df1 = pd.read_excel(file_path2)

file_path3 = '/content/drive/MyDrive/1-s2.0-S027323002200174X-mmc3.xlsx'
df2 = pd.read_excel(file_path3)

In [ ]:
# test acsess
print(df1.head())
print(df2.head())

In [ ]:
def fetch_abstract(url):
    # URL needs DOI or not
    if not url.startswith('http'):
        url = 'https://doi.org/' + url

    try:
        response = requests.get(url, allow_redirects=True, timeout=30)  # Increased timeout
        response.raise_for_status()

        content_type = response.headers.get('Content-Type', '').lower()

        if content_type.startswith('application/pdf'):
            # Skip PDFs and return None
            return None, False
        elif content_type.startswith('text/html'):
            # Extract text from HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # find the abstract using common patterns
            abstract = None

            # Check for common tags and classes/ids
            possible_selectors = [
                {'name': 'div', 'class_': 'abstract'},  # Example: <div class="abstract">
                {'name': 'section', 'class_': 'abstract'},  # Example: <section class="abstract">
                {'name': 'p', 'class_': 'abstract'},  # Example: <p class="abstract">
                {'name': 'div', 'id': 'abstract'},  # Example: <div id="abstract">
                {'name': 'section', 'id': 'abstract'},  # Example: <section id="abstract">
                {'name': 'p', 'id': 'abstract'},  # Example: <p id="abstract">
                {'name': 'div', 'class_': 'summary'},  # Example: <div class="summary">
                {'name': 'section', 'class_': 'summary'},  # Example: <section class="summary">
                {'name': 'p', 'class_': 'summary'},  # Example: <p class="summary">
                {'name': 'div', 'id': 'summary'},  # Example: <div id="summary">
                {'name': 'section', 'id': 'summary'},  # Example: <section id="summary">
                {'name': 'p', 'id': 'summary'},  # Example: <p id="summary">
                {'name': 'div', 'class_': 'overview'},  # Example: <div class="overview">
                {'name': 'section', 'class_': 'overview'},  # Example: <section class="overview">
                {'name': 'p', 'class_': 'overview'},  # Example: <p class="overview">
                {'name': 'div', 'id': 'overview'},  # Example: <div id="overview">
                {'name': 'section', 'id': 'overview'},  # Example: <section id="overview">
                {'name': 'p', 'id': 'overview'},  # Example: <p id="overview">
                {'name': 'div', 'class_': 'conclusion'},  # Example: <div class="conclusion">
                {'name': 'section', 'class_': 'conclusion'},  # Example: <section class="conclusion">
                {'name': 'p', 'class_': 'conclusion'},  # Example: <p class="conclusion">
                {'name': 'div', 'id': 'conclusion'},  # Example: <div id="conclusion">
                {'name': 'section', 'id': 'conclusion'},  # Example: <section id="conclusion">
                {'name': 'p', 'id': 'conclusion'},  # Example: <p id="conclusion">
                {'name': 'div', 'class_': 'highlights'},  # Example: <div class="highlights">
                {'name': 'section', 'class_': 'highlights'},  # Example: <section class="highlights">
                {'name': 'p', 'class_': 'highlights'},  # Example: <p class="highlights">
                {'name': 'div', 'id': 'highlights'},  # Example: <div id="highlights">
                {'name': 'section', 'id': 'highlights'},  # Example: <section id="highlights">
                {'name': 'p', 'id': 'highlights'},  # Example: <p id="highlights">
                {'name': 'div', 'class_': 'executive-summary'},  # Example: <div class="executive-summary">
                {'name': 'section', 'class_': 'executive-summary'},  # Example: <section class="executive-summary">
                {'name': 'p', 'class_': 'executive-summary'},  # Example: <p class="executive-summary">
                {'name': 'div', 'id': 'executive-summary'},  # Example: <div id="executive-summary">
                {'name': 'section', 'id': 'executive-summary'},  # Example: <section id="executive-summary">
                {'name': 'p', 'id': 'executive-summary'},  # Example: <p id="executive-summary">
                {'name': 'div', 'class_': 'c-article-section__content', 'id': 'Abs1-content'},  # Example: <div class="c-article-section__content" id="Abs1-content">
            ]

            for selector in possible_selectors:
                abstract_tag = soup.find(selector['name'], class_=selector.get('class_'), id=selector.get('id'))
                if abstract_tag:
                    abstract = ' '.join(abstract_tag.stripped_strings)  # Join all the strings in the tag, handling nested tags
                    break

            if abstract:
                return html.unescape(abstract), True  # Unescape HTML entities
            else:
                # Fallback: Try to find an abstract within meta tags (common in some articles)
                meta_abstract = soup.find('meta', attrs={'name': 'description'})
                if meta_abstract and meta_abstract.get('content'):
                    return html.unescape(meta_abstract['content']), True

                return None, True
        else:
            return None, False
    except requests.exceptions.RequestException as e:
        return None, False

def process_row(index, row):
    url = row['url']
    abstract = row['abstract']

    # Skip URLs that are empty, NaN, or contain "Internal Retrieval"
    if pd.isna(url) or not url or 'Internal Retrieval' in url:
        return None

    result = {
        'index': index,
        'url': url,
        'abstract': None,
        'is_html': False,
    }
    if abstract and not pd.isna(abstract):  # Check if 'abstract' is not empty
        fetched_abstract, is_html = fetch_abstract(url)
        if fetched_abstract:  # Only count if an actual abstract is returned
            result['abstract'] = fetched_abstract
        result['is_html'] = is_html
    return result

total = 0
abstracts_fetched = 0
urls_opened = 0
htmls_opened = 0

results = []

# Using ThreadPoolExecutor for concurrent requests
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_row, index, row) for index, row in df1.iterrows()]
    for future in as_completed(futures):
        result = future.result()
        if result is None:
            continue  # Skip this result if it was filtered out
        total += 1
        if result['abstract']:
            abstracts_fetched += 1
            print(f"URL: {result['url']}")
            print(f"Abstract: {result['abstract']}")
            print()
        if result['is_html']:
            htmls_opened += 1
            print(f"URL: {result['url']}")
            print()
        if result['url']:
            urls_opened += 1

print("Total iterations:", total)
print()
print("Abstracts fetched:", abstracts_fetched)
print()
print("URLs opened:", urls_opened)
print()
print("HTML pages opened:", htmls_opened)

In [ ]:
!pip install openai==0.28.0
import openai

In [ ]:
openai.api_key = 'X'